In [161]:
import classification_trees as trees
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
import itertools
from typing import Tuple
import time

In [4]:
def accuracy(y_test, y_pred):
    cf_matrix = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cf_matrix.ravel()
#     print(cf_matrix)

    accuracy = (tn + tp)/(tn + tp + fp + fn)
    print(f'Accuracy {accuracy:.2f}')
    return accuracy

## Classification trees, RF and Bagging 
Regular classification trees, RF and bagging comparison on the bug data set. The 2.0 set was used for training and the 3.0 set for testing.

*Regular Classification Tree*: nmin=15, minleaf=5, nfeats=41  
*Random Forest*: nmin=20, minleaf=5, nfeats=$\sqrt{41}$, ntrees=100  
*Bagging*: nmin=20, minleaf=5, nfeats=41, ntrees=100

In [5]:
pd.set_option('display.max_columns', None)

In [6]:
# Load training data (2.0)
train_data = pd.read_csv("bug-data/eclipse-metrics-packages-2.0.csv", delimiter=';')
train_data

,plugin,packagename,pre,post,ACD_avg,ACD_max,ACD_sum,FOUT_avg,FOUT_max,FOUT_sum,MLOC_avg,MLOC_max,MLOC_sum,NBD_avg,NBD_max,NBD_sum,NOCU,NOF_avg,NOF_max,NOF_sum,NOI_avg,NOI_max,NOI_sum,NOM_avg,NOM_max,NOM_sum,NOT_avg,NOT_max,NOT_sum,NSF_avg,NSF_max,NSF_sum,NSM_avg,NSM_max,NSM_sum,PAR_avg,PAR_max,PAR_sum,TLOC_avg,TLOC_max,TLOC_sum,VG_avg,VG_max,VG_sum,AnonymousClassDeclaration,ArrayAccess,ArrayCreation,ArrayInitializer,ArrayType,AssertStatement,Assignment,Block,BooleanLiteral,BreakStatement,CastExpression,CatchClause,CharacterLiteral,ClassInstanceCreation,CompilationUnit,ConditionalExpression,ConstructorInvocation,ContinueStatement,DoStatement,EmptyStatement,ExpressionStatement,FieldAccess,FieldDeclaration,ForStatement,IfStatement,ImportDeclaration,InfixExpression,Initializer,Javadoc,LabeledStatement,MethodDeclaration,MethodInvocation,NullLiteral,NumberLiteral,PackageDeclaration,ParenthesizedExpression,PostfixExpression,PrefixExpression,PrimitiveType,QualifiedName,ReturnStatement,SimpleName,SimpleType,SingleVariableDeclaration,StringLiteral,SuperConstructorInvocation,SuperFieldAccess,SuperMethodInvocation,SwitchCase,SwitchStatement,SynchronizedStatement,ThisExpression,ThrowStatement,TryStatement,TypeDeclaration,TypeDeclarationStatement,TypeLiteral,VariableDeclarationExpression,VariableDeclarationFragment,VariableDeclarationStatement,WhileStatement,InstanceofExpression,LineComment,BlockComment,TagElement,TextElement,MemberRef,MethodRef,MethodRefParameter,EnhancedForStatement,EnumDeclaration,EnumConstantDeclaration,TypeParameter,ParameterizedType,QualifiedType,WildcardType,NormalAnnotation,MarkerAnnotation,SingleMemberAnnotation,MemberValuePair,AnnotationTypeDeclaration,AnnotationTypeMemberDeclaration,Modifier,SUM,NORM_AnonymousClassDeclaration,NORM_ArrayAccess,NORM_ArrayCreation,NORM_ArrayInitializer,NORM_ArrayType,NORM_AssertStatement,NORM_Assignment,NORM_Block,NORM_BooleanLiteral,NORM_BreakStatement,NORM_CastExpression,NORM_CatchClause,NORM_CharacterLiteral,NORM_ClassInstanceCreation,NORM_CompilationUnit,NORM_ConditionalExpression,NORM_ConstructorInvocation,NORM_ContinueStatement,NORM_DoStatement,NORM_EmptyStatement,NORM_ExpressionStatement,NORM_FieldAccess,NORM_FieldDeclaration,NORM_ForStatement,NORM_IfStatement,NORM_ImportDeclaration,NORM_InfixExpression,NORM_Initializer,NORM_Javadoc,NORM_LabeledStatement,NORM_MethodDeclaration,NORM_MethodInvocation,NORM_NullLiteral,NORM_NumberLiteral,NORM_PackageDeclaration,NORM_ParenthesizedExpression,NORM_PostfixExpression,NORM_PrefixExpression,NORM_PrimitiveType,NORM_QualifiedName,NORM_ReturnStatement,NORM_SimpleName,NORM_SimpleType,NORM_SingleVariableDeclaration,NORM_StringLiteral,NORM_SuperConstructorInvocation,NORM_SuperFieldAccess,NORM_SuperMethodInvocation,NORM_SwitchCase,NORM_SwitchStatement,NORM_SynchronizedStatement,NORM_ThisExpression,NORM_ThrowStatement,NORM_TryStatement,NORM_TypeDeclaration,NORM_TypeDeclarationStatement,NORM_TypeLiteral,NORM_VariableDeclarationExpression,NORM_VariableDeclarationFragment,NORM_VariableDeclarationStatement,NORM_WhileStatement,NORM_InstanceofExpression,NORM_LineComment,NORM_BlockComment,NORM_TagElement,NORM_TextElement,NORM_MemberRef,NORM_MethodRef,NORM_MethodRefParameter,NORM_EnhancedForStatement,NORM_EnumDeclaration,NORM_EnumConstantDeclaration,NORM_TypeParameter,NORM_ParameterizedType,NORM_QualifiedType,NORM_WildcardType,NORM_NormalAnnotation,NORM_MarkerAnnotation,NORM_SingleMemberAnnotation,NORM_MemberValuePair,NORM_AnnotationTypeDeclaration,NORM_AnnotationTypeMemberDeclaration,NORM_Modifier
0,org.eclipse.jdt.junit,org.eclipse.jdt.internal.junit.launcher,5,0,0.571429,4.0,4.0,5.980769,29.0,311.0,9.230769,55.0,480.0,1.826923,6.0,95.0,7.0,1.250000,8.0,10.0,0.000000,0.0,0.0,6.375000,18.0,51.0,1.000000,1.0,7.0,0.500000,3.0,4.0,0.125000,1.0,1.0,1.173077,4.0,61.0,112.000000,277.0,784.0,2.576923,11.0,134.0,4,8,9,7,28,0,42,146,10,0,20,18,3,46,7,0,0,0,0,0,156,0,14,2,55,163,63,0,42,0,56,312,50,35,7,16,2,3,40,777,52,2349,294,83,49,2,0,6,0,0,0,0,1,14,8,0,2,

In [7]:
train_data = train_data.iloc[:, 2:44]
train_data['post'] = np.where(train_data['post'] > 0, 1, 0)

In [33]:
X_train, y_train = train_data.loc[:, ~train_data.columns.isin(['post'])], train_data['post']
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

377

In [34]:
# Load test data
test_data = pd.read_csv("bug-data/eclipse-metrics-packages-3.0.csv", delimiter=';')
test_data.shape

(661, 211)

In [35]:
test_data = test_data.iloc[:, 2:44]
test_data['post'] = np.where(test_data['post'] > 0, 1, 0)

In [47]:
X_test, y_test = test_data.loc[:, ~test_data.columns.isin(['post'])], test_data['post']
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

In [63]:
#Classification Tree
dt_clf = trees.ClassificationTree(nmin=15, minleaf=5)
dt_clf.fit(X_train, y_train, nfeats=41)
y_pred = dt_clf.predict(X_test)
print(f'Classification Tree accuracy: {accuracy(y_test, y_pred):.2f}')

Accuracy 0.69
Classification Tree accuracy: 0.69


In [76]:
#Random Forest
rf_clf = trees.RandomForestClassifier(nmin=15, minleaf=5)
rf_clf.fit(X_train, y_train)
y_pred = rf_clf.predict(X_test)
print(f'Random Forest accuracy: {accuracy(y_test, y_pred):.2f}')

Accuracy 0.77
Random Forest accuracy: 0.77


In [75]:
#Bagging
rf_clf = trees.RandomForestClassifier(nmin=15, minleaf=5)
rf_clf.fit(X_train, y_train, nfeats=41)
y_pred = rf_clf.predict(X_test)
print(f'Bagging accuracy: {accuracy(y_test, y_pred):.2f}')

Accuracy 0.79
Bagging accuracy: 0.79


## Hyperparameter optimization for RF using CV
An attempt to optimize hyperparameters was made for RF using 5-fold CV. The best hyperparameter combination resulting from the search was subsequently used to retrain on the training set, and performance of this model was assessed on the test set. To keep the search within a reasonable  time length, we reduced the parameter to search substantially. A ~48 hour search time was estimated beforehand based on all possible parameter combinations:

avg 100 trees trained/cv-fold -> takes ~3s per 100 trees -> 5 folds x 3s = 15s per CV run (1 function evaluation with 1 hyperparameter combination)  
12,000 hyperparameter combinations -> 12,000 x 15 = ~ 2 days

In [129]:
# Best hyperparameter search for RF

X = X_train  # X_train is 2.0 train set
y = y_train   # y_train is 2.0 train set

def find_best_hyperparams(params: Tuple[int, int, int, int]) -> Tuple[int, int, int, float]:
       
    nmin, minleaf, ntrees, nfeats = params
    print(nmin, minleaf, ntrees, nfeats)
    kf = KFold(n_splits=5)
    accuracies = [] # Accuracy on each test fold. With k = 5 we evaluate 5 trained models per param combination
    for train_idxs, test_idxs in kf.split(X):
        X_train, X_test, y_train, y_test = X[train_idxs], X[test_idxs], y[train_idxs], y[test_idxs]
        rf_clf = trees.RandomForestClassifier(ntrees=ntrees, nmin=nmin, minleaf=minleaf)
        rf_clf.fit(X_train, y_train, nfeats=nfeats)
        y_pred = rf_clf.predict(X_test)
        accuracies.append(accuracy(y_test, y_pred))
    
    return (nmin, minleaf, ntrees, nfeats, np.mean(accuracies))
    

nmin = range(2, 26) #2-25
minleaf = range(1, 11) #1-10
ntrees = [10, 20, 30, 40, 50, 100, 150, 200] # 8, ntrees = m (bootstrap samples to draw)
nfeats = range(1, 11) #1-10
cartesian_product = list(itertools.product(nmin, minleaf, ntrees, nfeats))

# Reduce the search by not considering minleaf * 2 > nmin (redundant because split cannot happen)
param_combinations = []
for prod in cartesian_product: 
    if not (prod[1] * 2) > prod[0]:
        param_combinations.append(prod)

print(len(param_combinations))
    
hyperparams = list(map(find_best_hyperparams, param_combinations))

12000
2 1 10 1
Accuracy 0.58
Accuracy 0.75
Accuracy 0.71
Accuracy 0.49
Accuracy 0.68
2 1 10 2
Accuracy 0.58
Accuracy 0.72
Accuracy 0.73
Accuracy 0.69
Accuracy 0.75
2 1 10 3
Accuracy 0.63
Accuracy 0.74
Accuracy 0.75
Accuracy 0.69
Accuracy 0.69
2 1 10 4
Accuracy 0.61
Accuracy 0.71
Accuracy 0.64
Accuracy 0.61
Accuracy 0.84
2 1 10 5
Accuracy 0.68
Accuracy 0.79
Accuracy 0.67
Accuracy 0.55
Accuracy 0.80
2 1 10 6
Accuracy 0.66
Accuracy 0.75
Accuracy 0.71
Accuracy 0.64
Accuracy 0.67
2 1 10 7
Accuracy 0.66
Accuracy 0.68
Accuracy 0.73
Accuracy 0.65
Accuracy 0.73
2 1 10 8
Accuracy 0.63
Accuracy 0.67
Accuracy 0.77
Accuracy 0.63
Accuracy 0.76
2 1 10 9
Accuracy 0.66
Accuracy 0.68
Accuracy 0.72
Accuracy 0.59
Accuracy 0.77
2 1 10 10
Accuracy 0.67
Accuracy 0.70
Accuracy 0.69
Accuracy 0.65
Accuracy 0.81
2 1 20 1
Accuracy 0.63
Accuracy 0.72
Accuracy 0.77
Accuracy 0.65
Accuracy 0.71
2 1 20 2
Accuracy 0.67
Accuracy 0.71
Accuracy 0.73
Accuracy 0.68
Accuracy 0.71
2 1 20 3
Accuracy 0.70
Accuracy 0.75
Accuracy

Accuracy 0.71
Accuracy 0.76
Accuracy 0.69
Accuracy 0.73
3 1 30 5
Accuracy 0.67
Accuracy 0.74
Accuracy 0.68
Accuracy 0.63
Accuracy 0.77
3 1 30 6
Accuracy 0.67
Accuracy 0.70
Accuracy 0.73
Accuracy 0.59
Accuracy 0.81
3 1 30 7
Accuracy 0.67
Accuracy 0.75
Accuracy 0.75
Accuracy 0.60
Accuracy 0.77
3 1 30 8
Accuracy 0.66
Accuracy 0.70
Accuracy 0.75
Accuracy 0.61
Accuracy 0.79
3 1 30 9
Accuracy 0.64
Accuracy 0.72
Accuracy 0.72
Accuracy 0.69
Accuracy 0.84
3 1 30 10
Accuracy 0.63
Accuracy 0.78
Accuracy 0.79
Accuracy 0.61
Accuracy 0.87
3 1 40 1
Accuracy 0.63
Accuracy 0.70
Accuracy 0.79
Accuracy 0.72
Accuracy 0.69
3 1 40 2
Accuracy 0.67
Accuracy 0.74
Accuracy 0.72
Accuracy 0.63
Accuracy 0.72
3 1 40 3
Accuracy 0.64
Accuracy 0.76
Accuracy 0.69
Accuracy 0.68
Accuracy 0.67
3 1 40 4
Accuracy 0.62
Accuracy 0.76
Accuracy 0.77
Accuracy 0.73
Accuracy 0.79
3 1 40 5
Accuracy 0.63
Accuracy 0.75
Accuracy 0.75
Accuracy 0.71
Accuracy 0.73
3 1 40 6
Accuracy 0.66
Accuracy 0.75
Accuracy 0.77
Accuracy 0.67
Accuracy 

Accuracy 0.61
Accuracy 0.83
4 1 50 8
Accuracy 0.66
Accuracy 0.76
Accuracy 0.76
Accuracy 0.72
Accuracy 0.85
4 1 50 9
Accuracy 0.68
Accuracy 0.76
Accuracy 0.75
Accuracy 0.64
Accuracy 0.76
4 1 50 10
Accuracy 0.64
Accuracy 0.80
Accuracy 0.77
Accuracy 0.60
Accuracy 0.83
4 1 100 1
Accuracy 0.66
Accuracy 0.74
Accuracy 0.81
Accuracy 0.71
Accuracy 0.68
4 1 100 2
Accuracy 0.66
Accuracy 0.75
Accuracy 0.77
Accuracy 0.65
Accuracy 0.68
4 1 100 3
Accuracy 0.66
Accuracy 0.72
Accuracy 0.77
Accuracy 0.77
Accuracy 0.76
4 1 100 4
Accuracy 0.64
Accuracy 0.76
Accuracy 0.75
Accuracy 0.69
Accuracy 0.75
4 1 100 5
Accuracy 0.68
Accuracy 0.74
Accuracy 0.79
Accuracy 0.67
Accuracy 0.80
4 1 100 6
Accuracy 0.63
Accuracy 0.78
Accuracy 0.77
Accuracy 0.60
Accuracy 0.79
4 1 100 7
Accuracy 0.68
Accuracy 0.74
Accuracy 0.79
Accuracy 0.63
Accuracy 0.75
4 1 100 8
Accuracy 0.70
Accuracy 0.78
Accuracy 0.80
Accuracy 0.64
Accuracy 0.81
4 1 100 9
Accuracy 0.68
Accuracy 0.76
Accuracy 0.79
Accuracy 0.63
Accuracy 0.79
4 1 100 10
Acc

Accuracy 0.64
Accuracy 0.84
4 2 200 1
Accuracy 0.67
Accuracy 0.72
Accuracy 0.77
Accuracy 0.72
Accuracy 0.75
4 2 200 2
Accuracy 0.66
Accuracy 0.74
Accuracy 0.76
Accuracy 0.77
Accuracy 0.73
4 2 200 3
Accuracy 0.64
Accuracy 0.76
Accuracy 0.77
Accuracy 0.75
Accuracy 0.72
4 2 200 4
Accuracy 0.66
Accuracy 0.75
Accuracy 0.76
Accuracy 0.73
Accuracy 0.75
4 2 200 5
Accuracy 0.66
Accuracy 0.76
Accuracy 0.77
Accuracy 0.71
Accuracy 0.79
4 2 200 6
Accuracy 0.68
Accuracy 0.76
Accuracy 0.75
Accuracy 0.69
Accuracy 0.79
4 2 200 7
Accuracy 0.66
Accuracy 0.78
Accuracy 0.75
Accuracy 0.63
Accuracy 0.80
4 2 200 8
Accuracy 0.70
Accuracy 0.72
Accuracy 0.76
Accuracy 0.63
Accuracy 0.85
4 2 200 9
Accuracy 0.68
Accuracy 0.76
Accuracy 0.79
Accuracy 0.61
Accuracy 0.85
4 2 200 10
Accuracy 0.66
Accuracy 0.78
Accuracy 0.79
Accuracy 0.59
Accuracy 0.81
5 1 10 1
Accuracy 0.62
Accuracy 0.61
Accuracy 0.72
Accuracy 0.65
Accuracy 0.81
5 1 10 2
Accuracy 0.67
Accuracy 0.75
Accuracy 0.61
Accuracy 0.60
Accuracy 0.80
5 1 10 3
Accu

Accuracy 0.61
5 2 20 4
Accuracy 0.64
Accuracy 0.70
Accuracy 0.76
Accuracy 0.65
Accuracy 0.68
5 2 20 5
Accuracy 0.64
Accuracy 0.68
Accuracy 0.69
Accuracy 0.63
Accuracy 0.73
5 2 20 6
Accuracy 0.64
Accuracy 0.75
Accuracy 0.77
Accuracy 0.68
Accuracy 0.77
5 2 20 7
Accuracy 0.67
Accuracy 0.71
Accuracy 0.75
Accuracy 0.60
Accuracy 0.77
5 2 20 8
Accuracy 0.64
Accuracy 0.66
Accuracy 0.75
Accuracy 0.61
Accuracy 0.80
5 2 20 9
Accuracy 0.68
Accuracy 0.74
Accuracy 0.69
Accuracy 0.63
Accuracy 0.76
5 2 20 10
Accuracy 0.63
Accuracy 0.82
Accuracy 0.80
Accuracy 0.56
Accuracy 0.83
5 2 30 1
Accuracy 0.61
Accuracy 0.67
Accuracy 0.76
Accuracy 0.72
Accuracy 0.69
5 2 30 2
Accuracy 0.62
Accuracy 0.71
Accuracy 0.69
Accuracy 0.73
Accuracy 0.77
5 2 30 3
Accuracy 0.67
Accuracy 0.68
Accuracy 0.69
Accuracy 0.69
Accuracy 0.73
5 2 30 4
Accuracy 0.66
Accuracy 0.76
Accuracy 0.73
Accuracy 0.68
Accuracy 0.71
5 2 30 5
Accuracy 0.63
Accuracy 0.78
Accuracy 0.79
Accuracy 0.73
Accuracy 0.84
5 2 30 6
Accuracy 0.68
Accuracy 0.74


...

Accuracy 0.72
Accuracy 0.67
Accuracy 0.69
25 5 20 2
Accuracy 0.63
Accuracy 0.71
Accuracy 0.76
Accuracy 0.81
Accuracy 0.61
25 5 20 3
Accuracy 0.68
Accuracy 0.68
Accuracy 0.73
Accuracy 0.75
Accuracy 0.71
25 5 20 4
Accuracy 0.64
Accuracy 0.68
Accuracy 0.72
Accuracy 0.68
Accuracy 0.64
25 5 20 5
Accuracy 0.64
Accuracy 0.75
Accuracy 0.71
Accuracy 0.68
Accuracy 0.71
25 5 20 6
Accuracy 0.68
Accuracy 0.76
Accuracy 0.75
Accuracy 0.69
Accuracy 0.71
25 5 20 7
Accuracy 0.67
Accuracy 0.72
Accuracy 0.77
Accuracy 0.73
Accuracy 0.67
25 5 20 8
Accuracy 0.64
Accuracy 0.64
Accuracy 0.77
Accuracy 0.69
Accuracy 0.72
25 5 20 9
Accuracy 0.66
Accuracy 0.72
Accuracy 0.73
Accuracy 0.55
Accuracy 0.81
25 5 20 10
Accuracy 0.71
Accuracy 0.72
Accuracy 0.81
Accuracy 0.69
Accuracy 0.73
25 5 30 1
Accuracy 0.64
Accuracy 0.71
Accuracy 0.69
Accuracy 0.79
Accuracy 0.73
25 5 30 2
Accuracy 0.66
Accuracy 0.75
Accuracy 0.80
Accuracy 0.65
Accuracy 0.72
25 5 30 3
Accuracy 0.67
Accuracy 0.74
Accuracy 0.76
Accuracy 0.81
Accuracy 0.

Accuracy 0.75
Accuracy 0.72
Accuracy 0.69
25 6 40 4
Accuracy 0.63
Accuracy 0.72
Accuracy 0.75
Accuracy 0.67
Accuracy 0.69
25 6 40 5
Accuracy 0.64
Accuracy 0.72
Accuracy 0.76
Accuracy 0.75
Accuracy 0.72
25 6 40 6
Accuracy 0.64
Accuracy 0.71
Accuracy 0.72
Accuracy 0.60
Accuracy 0.80
25 6 40 7
Accuracy 0.67
Accuracy 0.71
Accuracy 0.76
Accuracy 0.67
Accuracy 0.76
25 6 40 8
Accuracy 0.68
Accuracy 0.70
Accuracy 0.75
Accuracy 0.76
Accuracy 0.79
25 6 40 9
Accuracy 0.71
Accuracy 0.78
Accuracy 0.80
Accuracy 0.60
Accuracy 0.85
25 6 40 10
Accuracy 0.67
Accuracy 0.78
Accuracy 0.76
Accuracy 0.64
Accuracy 0.63
25 6 50 1
Accuracy 0.66
Accuracy 0.70
Accuracy 0.79
Accuracy 0.71
Accuracy 0.65
25 6 50 2
Accuracy 0.68
Accuracy 0.68
Accuracy 0.80
Accuracy 0.68
Accuracy 0.68
25 6 50 3
Accuracy 0.70
Accuracy 0.76
Accuracy 0.73
Accuracy 0.76
Accuracy 0.72
25 6 50 4
Accuracy 0.70
Accuracy 0.74
Accuracy 0.76
Accuracy 0.65
Accuracy 0.77
25 6 50 5
Accuracy 0.67
Accuracy 0.68
Accuracy 0.79
Accuracy 0.68
Accuracy 0.

Accuracy 0.73
Accuracy 0.75
Accuracy 0.75
25 7 100 6
Accuracy 0.68
Accuracy 0.71
Accuracy 0.76
Accuracy 0.65
Accuracy 0.71
25 7 100 7
Accuracy 0.63
Accuracy 0.74
Accuracy 0.79
Accuracy 0.73
Accuracy 0.83
25 7 100 8
Accuracy 0.66
Accuracy 0.74
Accuracy 0.76
Accuracy 0.69
Accuracy 0.79
25 7 100 9
Accuracy 0.70
Accuracy 0.75
Accuracy 0.79
Accuracy 0.68
Accuracy 0.77
25 7 100 10
Accuracy 0.67
Accuracy 0.75
Accuracy 0.76
Accuracy 0.69
Accuracy 0.83
25 7 150 1
Accuracy 0.70
Accuracy 0.75
Accuracy 0.80
Accuracy 0.75
Accuracy 0.69
25 7 150 2
Accuracy 0.67
Accuracy 0.70
Accuracy 0.77
Accuracy 0.73
Accuracy 0.67
25 7 150 3
Accuracy 0.68
Accuracy 0.72
Accuracy 0.77
Accuracy 0.65
Accuracy 0.75
25 7 150 4
Accuracy 0.67
Accuracy 0.75
Accuracy 0.77
Accuracy 0.67
Accuracy 0.73
25 7 150 5
Accuracy 0.66
Accuracy 0.71
Accuracy 0.77
Accuracy 0.76
Accuracy 0.71
25 7 150 6
Accuracy 0.67
Accuracy 0.74
Accuracy 0.77
Accuracy 0.76
Accuracy 0.76
25 7 150 7
Accuracy 0.67
Accuracy 0.76
Accuracy 0.76
Accuracy 0.69

Accuracy 0.70
Accuracy 0.74
Accuracy 0.72
Accuracy 0.77
Accuracy 0.73
25 8 200 8
Accuracy 0.67
Accuracy 0.75
Accuracy 0.77
Accuracy 0.68
Accuracy 0.76
25 8 200 9
Accuracy 0.70
Accuracy 0.75
Accuracy 0.75
Accuracy 0.61
Accuracy 0.77
25 8 200 10
Accuracy 0.68
Accuracy 0.75
Accuracy 0.77
Accuracy 0.67
Accuracy 0.83
25 9 10 1
Accuracy 0.72
Accuracy 0.68
Accuracy 0.73
Accuracy 0.65
Accuracy 0.60
25 9 10 2
Accuracy 0.59
Accuracy 0.70
Accuracy 0.77
Accuracy 0.63
Accuracy 0.67
25 9 10 3
Accuracy 0.59
Accuracy 0.76
Accuracy 0.73
Accuracy 0.73
Accuracy 0.65
25 9 10 4
Accuracy 0.62
Accuracy 0.68
Accuracy 0.77
Accuracy 0.77
Accuracy 0.79
25 9 10 5
Accuracy 0.71
Accuracy 0.67
Accuracy 0.69
Accuracy 0.63
Accuracy 0.68
25 9 10 6
Accuracy 0.61
Accuracy 0.70
Accuracy 0.64
Accuracy 0.61
Accuracy 0.84
25 9 10 7
Accuracy 0.59
Accuracy 0.67
Accuracy 0.72
Accuracy 0.69
Accuracy 0.64
25 9 10 8
Accuracy 0.62
Accuracy 0.74
Accuracy 0.73
Accuracy 0.65
Accuracy 0.68
25 9 10 9
Accuracy 0.68
Accuracy 0.76
Accuracy

Accuracy 0.80
25 10 20 9
Accuracy 0.66
Accuracy 0.71
Accuracy 0.75
Accuracy 0.64
Accuracy 0.81
25 10 20 10
Accuracy 0.68
Accuracy 0.79
Accuracy 0.73
Accuracy 0.76
Accuracy 0.72
25 10 30 1
Accuracy 0.68
Accuracy 0.74
Accuracy 0.77
Accuracy 0.72
Accuracy 0.72
25 10 30 2
Accuracy 0.71
Accuracy 0.71
Accuracy 0.76
Accuracy 0.76
Accuracy 0.69
25 10 30 3
Accuracy 0.66
Accuracy 0.70
Accuracy 0.71
Accuracy 0.79
Accuracy 0.65
25 10 30 4
Accuracy 0.68
Accuracy 0.66
Accuracy 0.76
Accuracy 0.72
Accuracy 0.80
25 10 30 5
Accuracy 0.70
Accuracy 0.68
Accuracy 0.83
Accuracy 0.63
Accuracy 0.71
25 10 30 6
Accuracy 0.66
Accuracy 0.72
Accuracy 0.79
Accuracy 0.73
Accuracy 0.69
25 10 30 7
Accuracy 0.68
Accuracy 0.72
Accuracy 0.80
Accuracy 0.73
Accuracy 0.81
25 10 30 8
Accuracy 0.74
Accuracy 0.82
Accuracy 0.77
Accuracy 0.60
Accuracy 0.79
25 10 30 9
Accuracy 0.74
Accuracy 0.76
Accuracy 0.73
Accuracy 0.55
Accuracy 0.79
25 10 30 10
Accuracy 0.70
Accuracy 0.76
Accuracy 0.72
Accuracy 0.61
Accuracy 0.73
25 10 40 1
A

In [130]:
max(hyperparams, key=lambda x: x[4])

(25, 6, 30, 8, 0.7827368421052632)

In [146]:
df = pd.DataFrame(hyperparams, columns=['nmin', 'minleaf', 'ntrees', 'nfeats', 'avg_accuracy'])
df.to_csv('hyperparams.csv')

In [163]:
df_sorted = df.sort_values(by='avg_accuracy', ascending=False).head(5)
df_sorted

,nmin,minleaf,ntrees,nfeats,avg_accuracy
11627,25,6,30,8,0.782737
5073,16,8,40,4,0.780035
5445,17,5,10,6,0.777333
8729,21,10,10,10,0.769263
3868,14,7,30,9,0.766912


In [165]:
df_sorted.nmin.unique()

array([25, 16, 17, 21, 14])

In [166]:
df_sorted.minleaf.unique()

array([ 6,  8,  5, 10,  7])

In [167]:
df_sorted.nfeats.unique()

array([ 8,  4,  6, 10,  9])

In [168]:
# Do also a search for 300, 400 and 500 trees, but only for top 5 nmin, minleaf and nfeats from previous search
nmin = [25, 16, 17, 21, 14]
minleaf = [6,  8,  5, 10,  7]
ntrees = [300, 400, 500]
nfeats = [8,  4,  6, 10,  9]
cartesian_product = list(itertools.product(nmin, minleaf, ntrees, nfeats))

param_combinations = []
for prod in cartesian_product: 
    if not (prod[1] * 2) > prod[0]:
        param_combinations.append(prod)

print(len(param_combinations))

315


In [170]:
t1 = time.time()
hyperparams2 = list(map(find_best_hyperparams, param_combinations))
t2 = time.time()
print(f'Took: {t2-t1:.2f}s')

25 6 300 8
Accuracy 0.70
Accuracy 0.72
Accuracy 0.77
Accuracy 0.68
Accuracy 0.77
25 6 300 4
Accuracy 0.68
Accuracy 0.72
Accuracy 0.77
Accuracy 0.77
Accuracy 0.72
25 6 300 6
Accuracy 0.70
Accuracy 0.74
Accuracy 0.77
Accuracy 0.69
Accuracy 0.73
25 6 300 10
Accuracy 0.68
Accuracy 0.76
Accuracy 0.77
Accuracy 0.64
Accuracy 0.84
25 6 300 9
Accuracy 0.68
Accuracy 0.75
Accuracy 0.76
Accuracy 0.71
Accuracy 0.80
25 6 400 8
Accuracy 0.70
Accuracy 0.75
Accuracy 0.75
Accuracy 0.73
Accuracy 0.80
25 6 400 4
Accuracy 0.67
Accuracy 0.74
Accuracy 0.79
Accuracy 0.75
Accuracy 0.72
25 6 400 6
Accuracy 0.67
Accuracy 0.74
Accuracy 0.79
Accuracy 0.68
Accuracy 0.68
25 6 400 10
Accuracy 0.70
Accuracy 0.75
Accuracy 0.77
Accuracy 0.64
Accuracy 0.77
25 6 400 9
Accuracy 0.68
Accuracy 0.74
Accuracy 0.77
Accuracy 0.68
Accuracy 0.81
25 6 500 8
Accuracy 0.68
Accuracy 0.75
Accuracy 0.77
Accuracy 0.68
Accuracy 0.79
25 6 500 4
Accuracy 0.67
Accuracy 0.74
Accuracy 0.76
Accuracy 0.77
Accuracy 0.72
25 6 500 6
Accuracy 0.70
A

Accuracy 0.80
16 8 500 4
Accuracy 0.68
Accuracy 0.74
Accuracy 0.77
Accuracy 0.76
Accuracy 0.72
16 8 500 6
Accuracy 0.67
Accuracy 0.72
Accuracy 0.77
Accuracy 0.76
Accuracy 0.75
16 8 500 10
Accuracy 0.68
Accuracy 0.74
Accuracy 0.77
Accuracy 0.63
Accuracy 0.81
16 8 500 9
Accuracy 0.68
Accuracy 0.75
Accuracy 0.76
Accuracy 0.67
Accuracy 0.80
16 5 300 8
Accuracy 0.67
Accuracy 0.78
Accuracy 0.77
Accuracy 0.65
Accuracy 0.80
16 5 300 4
Accuracy 0.67
Accuracy 0.75
Accuracy 0.77
Accuracy 0.71
Accuracy 0.73
16 5 300 6
Accuracy 0.66
Accuracy 0.75
Accuracy 0.77
Accuracy 0.68
Accuracy 0.80
16 5 300 10
Accuracy 0.70
Accuracy 0.75
Accuracy 0.77
Accuracy 0.64
Accuracy 0.81
16 5 300 9
Accuracy 0.68
Accuracy 0.78
Accuracy 0.76
Accuracy 0.68
Accuracy 0.76
16 5 400 8
Accuracy 0.68
Accuracy 0.76
Accuracy 0.77
Accuracy 0.71
Accuracy 0.80
16 5 400 4
Accuracy 0.67
Accuracy 0.74
Accuracy 0.79
Accuracy 0.73
Accuracy 0.72
16 5 400 6
Accuracy 0.67
Accuracy 0.75
Accuracy 0.76
Accuracy 0.64
Accuracy 0.79
16 5 400 10


Accuracy 0.73
21 6 400 6
Accuracy 0.66
Accuracy 0.75
Accuracy 0.76
Accuracy 0.71
Accuracy 0.77
21 6 400 10
Accuracy 0.68
Accuracy 0.76
Accuracy 0.77
Accuracy 0.65
Accuracy 0.80
21 6 400 9
Accuracy 0.68
Accuracy 0.74
Accuracy 0.76
Accuracy 0.63
Accuracy 0.81
21 6 500 8
Accuracy 0.68
Accuracy 0.75
Accuracy 0.76
Accuracy 0.68
Accuracy 0.81
21 6 500 4
Accuracy 0.66
Accuracy 0.75
Accuracy 0.75
Accuracy 0.76
Accuracy 0.72
21 6 500 6
Accuracy 0.68
Accuracy 0.74
Accuracy 0.77
Accuracy 0.69
Accuracy 0.72
21 6 500 10
Accuracy 0.70
Accuracy 0.74
Accuracy 0.77
Accuracy 0.65
Accuracy 0.80
21 6 500 9
Accuracy 0.68
Accuracy 0.75
Accuracy 0.75
Accuracy 0.68
Accuracy 0.83
21 8 300 8
Accuracy 0.68
Accuracy 0.74
Accuracy 0.76
Accuracy 0.65
Accuracy 0.77
21 8 300 4
Accuracy 0.67
Accuracy 0.74
Accuracy 0.79
Accuracy 0.71
Accuracy 0.71
21 8 300 6
Accuracy 0.67
Accuracy 0.75
Accuracy 0.77
Accuracy 0.72
Accuracy 0.76
21 8 300 10
Accuracy 0.67
Accuracy 0.74
Accuracy 0.77
Accuracy 0.63
Accuracy 0.81
21 8 300 9


Accuracy 0.71
Accuracy 0.75
14 7 300 10
Accuracy 0.70
Accuracy 0.74
Accuracy 0.77
Accuracy 0.67
Accuracy 0.83
14 7 300 9
Accuracy 0.67
Accuracy 0.74
Accuracy 0.79
Accuracy 0.68
Accuracy 0.83
14 7 400 8
Accuracy 0.70
Accuracy 0.74
Accuracy 0.77
Accuracy 0.69
Accuracy 0.79
14 7 400 4
Accuracy 0.67
Accuracy 0.74
Accuracy 0.77
Accuracy 0.71
Accuracy 0.69
14 7 400 6
Accuracy 0.67
Accuracy 0.75
Accuracy 0.76
Accuracy 0.67
Accuracy 0.73
14 7 400 10
Accuracy 0.68
Accuracy 0.75
Accuracy 0.79
Accuracy 0.60
Accuracy 0.81
14 7 400 9
Accuracy 0.70
Accuracy 0.75
Accuracy 0.77
Accuracy 0.67
Accuracy 0.81
14 7 500 8
Accuracy 0.68
Accuracy 0.72
Accuracy 0.79
Accuracy 0.71
Accuracy 0.81
14 7 500 4
Accuracy 0.66
Accuracy 0.72
Accuracy 0.77
Accuracy 0.75
Accuracy 0.76
14 7 500 6
Accuracy 0.67
Accuracy 0.74
Accuracy 0.77
Accuracy 0.77
Accuracy 0.77
14 7 500 10
Accuracy 0.68
Accuracy 0.75
Accuracy 0.76
Accuracy 0.68
Accuracy 0.84
14 7 500 9
Accuracy 0.67
Accuracy 0.75
Accuracy 0.76
Accuracy 0.65
Accuracy 0.

In [171]:
max(hyperparams2, key=lambda x: x[4])

(21, 5, 300, 10, 0.7587719298245614)

No improvement when increasing ntrees, for the given best hyperparameters from previous search

### Train RF classifier with best hyperparams that resulted from CV, and test accuracy on validation set

In [230]:
# Retrain on whole training set
rf_clf = trees.RandomForestClassifier(ntrees=30, nmin=25, minleaf=6)
rf_clf.fit(X_train, y_train)

In [231]:
y_pred = rf_clf.predict(X_test)
accuracy(y_test, y_pred)

Accuracy 0.80


0.7957639939485628

No major improvement in accuracy after hyperparameter optimization attempt